## Configs

In [6]:
offset = 0
limit = 100
period = '10y' # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max

## Download all NASDAQ traded symbols

In [7]:
import pandas as pd

data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N']
symbols = data_clean['NASDAQ Symbol'].tolist()
print('total number of symbols traded = {}'.format(len(symbols)))

total number of symbols traded = 11305


## Download Historic data

In [8]:
! pip install yfinance > /dev/null 2>&1
! mkdir hist

mkdir: hist: File exists


In [9]:
import yfinance as yf
import os, contextlib

In [10]:
# Single Stock
s = "AAPL"
folder = "test_stocks"
os.makedirs(folder, exist_ok=True) 

data = yf.download(s, period='max', auto_adjust=False)

if len(data.index) > 0:
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.get_level_values(0)

    filename = os.path.join(folder, f"{s}.csv")
    data.to_csv(filename)
    print(f"✔ Downloaded {s} to {filename}")
else:
    print(f"✘ No data found for {s}")

[*********************100%***********************]  1 of 1 completed

✔ Downloaded AAPL to test_stocks/AAPL.csv


In [11]:
%%time

limit = limit if limit else len(symbols)
end = min(offset + limit, len(symbols))
is_valid = [False] * len(symbols)

# force silencing of verbose API
with open(os.devnull, 'w') as devnull:
    with contextlib.redirect_stdout(devnull):
        for i in range(offset, end):
            s = symbols[i]
            data = yf.download(s, period=period, auto_adjust=False)
            if len(data.index) == 0:
                continue
            
            if isinstance(data.columns, pd.MultiIndex):
                data.columns = data.columns.get_level_values(0)
        
            is_valid[i] = True
            data.to_csv('hist/{}.csv'.format(s))

print('Total number of valid symbols downloaded = {}'.format(sum(is_valid)))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AACT=']: HTTPError('HTTP Error 404: ')
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AACT+']: HTTPError('HTTP Error 404: ')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]

Total number of valid symbols downloaded = 92
CPU times: user 4.84 s, sys: 656 ms, total: 5.5 s
Wall time: 54.5 s


In [ ]:
valid_data = data_clean[is_valid]
# valid_data.to_csv('symbols_valid_meta.csv', index=False)

## Separating ETFs and Stocks

In [13]:
!mkdir stocks
!mkdir etfs

mkdir: stocks: File exists
mkdir: etfs: File exists


In [14]:
etfs = valid_data[valid_data['ETF'] == 'Y']['NASDAQ Symbol'].tolist()
stocks = valid_data[valid_data['ETF'] == 'N']['NASDAQ Symbol'].tolist()

In [15]:
import shutil
from os.path import isfile, join

def move_symbols(symbols, dest):
    for s in symbols:
        filename = '{}.csv'.format(s)
        shutil.move(join('hist', filename), join(dest, filename))
        
move_symbols(etfs, "etfs")
move_symbols(stocks, "stocks")

In [16]:
! rmdir hist